<a href="https://colab.research.google.com/github/Engineer-Ayesha-Shafique/Brain-Tumor-Segmentation-and-Detection-using-UNET-and-Watershed-in-Python/blob/main/Brain_Tumor_Segmentation_and_Detection_using_UNET_and_Watershed_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/drive' ,force_remount=True)

ValueError: Mountpoint must not already contain files

#UNET



In [ ]:
!pip install tensorflow
!pip install --quiet segmentation-models tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import tensorflow as tf
import keras
import segmentation_models as sm
from segmentation_models import Unet, losses, metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, CSVLogger,
                                        ReduceLROnPlateau, EarlyStopping,
                                        TensorBoard)

# Tell segmentation-models to use tf.keras as its backend
sm.set_framework('tf.keras')
sm.framework()



import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation,
    MaxPool2D, UpSampling2D, Concatenate,
    GlobalAveragePooling2D, Reshape, Multiply, Add
)
from tensorflow.keras.models import Model

def conv_block(x, filters):
    x = Conv2D(filters, 3, padding='same', use_bias=True)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, 3, padding='same', use_bias=True)(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x)

def up_conv(x, filters):
    x = UpSampling2D()(x)
    x = Conv2D(filters, 3, padding='same', use_bias=True)(x)
    x = BatchNormalization()(x)
    return Activation('relu')(x)

def Recurrent_block(x, filters, t=2):
    for i in range(t):
        if i == 0:
            x1 = Conv2D(filters, 3, padding='same', use_bias=True)(x)
            x1 = BatchNormalization()(x1)
            x1 = Activation('relu')(x1)
        x1 = Conv2D(filters, 3, padding='same', use_bias=True)(Add()([x, x1]))
        x1 = BatchNormalization()(x1)
        x1 = Activation('relu')(x1)
    return x1

def RRCNN_block(x, filters, t=2):
    x_in = Conv2D(filters, 1, padding='same', use_bias=True)(x)
    x1 = Recurrent_block(x_in, filters, t=t)
    x1 = Recurrent_block(x1,  filters, t=t)
    return Add()([x_in, x1])

def Attention_block(g, x, inter_filters):
    # g: gating signal (decoder feature), x: skip connection feature
    g1 = Conv2D(inter_filters, 1, padding='same', use_bias=True)(g)
    g1 = BatchNormalization()(g1)
    x1 = Conv2D(inter_filters, 1, padding='same', use_bias=True)(x)
    x1 = BatchNormalization()(x1)
    psi = Activation('relu')(Add()([g1, x1]))
    psi = Conv2D(1, 1, padding='same', use_bias=True)(psi)
    psi = BatchNormalization()(psi)
    psi = Activation('sigmoid')(psi)
    return Multiply()([x, psi])

def build_r2attunet(input_shape, n_classes, t=2):
    inp = Input(input_shape)

    # Encoder
    c1 = conv_block(inp,   64)
    p1 = MaxPool2D()(c1)
    c2 = conv_block(p1,  128)
    p2 = MaxPool2D()(c2)
    c3 = conv_block(p2,  256)
    p3 = MaxPool2D()(c3)
    c4 = conv_block(p3,  512)
    p4 = MaxPool2D()(c4)
    c5 = conv_block(p4, 1024)

    # Decoder with Recurrent + Attention
    d5 = up_conv(c5, 512)
    a5 = Attention_block(d5, c4, 256)
    d5 = Concatenate()([a5, d5])
    d5 = RRCNN_block(d5, 512, t)

    d4 = up_conv(d5, 256)
    a4 = Attention_block(d4, c3, 128)
    d4 = Concatenate()([a4, d4])
    d4 = RRCNN_block(d4, 256, t)

    d3 = up_conv(d4, 128)
    a3 = Attention_block(d3, c2,  64)
    d3 = Concatenate()([a3, d3])
    d3 = RRCNN_block(d3, 128, t)

    d2 = up_conv(d3,  64)
    a2 = Attention_block(d2, c1,  32)
    d2 = Concatenate()([a2, d2])
    d2 = RRCNN_block(d2,  64, t)

    # Output
    out = Conv2D(n_classes, 1, padding='same', activation='softmax')(d2)

    return Model(inp, out, name="R2AttU_Net")

# ─── Usage ────────────────────────────────────────────────────────────
NUM_CLASSES  = 5
INPUT_SHAPE  = (224, 224, 3)
model = build_r2attunet(INPUT_SHAPE, NUM_CLASSES, t=2)
model.summary()


Segmentation Models: using `tf.keras` framework.


Model: "R2AttU_Net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 224, 224,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 224, 224,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_4[0][0]  

 Total params: 45,011,257 (171.70 MB)

 Trainable params: 44,987,953 (171.62 MB)

 Non-trainable params: 23,304 (91.03 KB)

In [ ]:
# install visualkeras for model plotting
!pip install --quiet visualkeras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.4/997.4 kB 17.4 MB/s eta 0:00:00


# Metrics

In [ ]:
import tensorflow as tf

smooth = 1e-15
num_classes = 5  # <-- make this match your final Conv2D(filters=5, ...)

def dice_coefficient(y_true, y_pred):
    # y_true: (B, H, W, 1) integers 0..4
    # y_pred: (B, H, W, 5) softmax probs

    # 1) Squeeze off the last dim → (B, H, W)
    y_true = tf.squeeze(y_true, axis=-1)

    # 2) One‑hot encode → (B, H, W, 5)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)

    # 3) Flatten both to (B*H*W, 5)
    y_true_f = tf.reshape(y_true_o, [-1, num_classes])
    y_pred_f = tf.reshape(y_pred,   [-1, num_classes])

    # 4) Compute intersection & union per class
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union        = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)

    # 5) Dice per class, then average
    dice_per_class = (2. * intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_coefficient_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)


#Train


In [ ]:
# In a notebook cell, before any cv2 imports:
!pip install --quiet opencv-python-headless
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 1) Import TF first and set memory growth (fallback to experimental)
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
        except Exception as e:
            print(f"[GPU] Could not set memory growth: {e}")
    print(f"[GPU] Detected {len(gpus)} GPU(s).")

# 2) Now import the rest
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import (
    ModelCheckpoint, CSVLogger, ReduceLROnPlateau,
    EarlyStopping, TensorBoard
)
from tensorflow.keras.optimizers import Adam

""" Global parameters """
Height = 512
Width  = 512

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path_dataset, val_ratio=0.2, test_ratio=0.1, random_state=42):
    img_dir  = os.path.join(path_dataset, "images")
    mask_dir = os.path.join(path_dataset, "masks")

    image_exts = ('.png', '.jpg', '.jpeg')
    images = sorted([
        os.path.join(img_dir, f)
        for f in os.listdir(img_dir)
        if f.lower().endswith(image_exts)
    ])


    mask_exts = ('.tif', '.tiff')
    masks = sorted([
        os.path.join(mask_dir, f)
        for f in os.listdir(mask_dir)
        if f.lower().endswith(mask_exts)
    ])

    if not images:
        raise RuntimeError(f"No images found in {img_dir}")
    if not masks:
        raise RuntimeError(f"No masks found in {mask_dir}")

    images_lookup = {
        os.path.splitext(os.path.basename(p))[0]: p
        for p in images
    }
#MildDemented_0a7b1321-eba0-40dc-85b8-de34241554a2_jpg.rf.70370fcd9ac0fb82f91d27f558053d84.tif
    pairs = []
    for m in masks:
        base = os.path.splitext(os.path.basename(m))[0]
        if base.endswith('_mask'):
            root = base[:-5]
            img_path = images_lookup.get(root)
            if img_path:
                pairs.append((img_path, m))

    if not pairs:
        raise RuntimeError("No matching image↔mask pairs found!")

    trainval, test = train_test_split(pairs, test_size=test_ratio, random_state=random_state)
    val_frac = val_ratio / (1.0 - test_ratio)
    train, val = train_test_split(trainval, test_size=val_frac, random_state=random_state)

    train_imgs, train_masks = zip(*train)
    val_imgs,   val_masks   = zip(*val)
    test_imgs,  test_masks  = zip(*test)

    return (
        list(train_imgs), list(train_masks),
        list(val_imgs),   list(val_masks),
        list(test_imgs),  list(test_masks)
    )

def read_image_file(path_img):
    path = path_img.decode('utf-8') if isinstance(path_img, bytes) else path_img
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (Width, Height)) / 255.0
    return img.astype(np.float32)

def read_mask_file(path_mask):
    path = path_mask.decode('utf-8') if isinstance(path_mask, bytes) else path_mask
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if mask.ndim == 3:
        mask = mask[..., 0]
    mask = cv2.resize(mask, (Width, Height), interpolation=cv2.INTER_NEAREST)

    # ✅ Clip to [0, 4], not [0, 3]:
    mask = np.clip(mask, 0, NUM_CLASSES-1)

    # Convert to integer class IDs
    mask = mask.astype(np.int32)

    # Add the channel dim back
    return mask[..., None]

def tf_parse(path_img, path_mask):
    # Tell numpy_function: img→float32, mask→int32
    img, mask = tf.numpy_function(
        lambda pi, pm: (read_image_file(pi), read_mask_file(pm)),
        [path_img, path_mask],
        [tf.float32, tf.int32]          # <-- second output is now int32
    )
    img.set_shape([Height, Width, 3])
    mask.set_shape([Height, Width, 1])
    return img, mask

def tf_dataset(inputs, targets, batch_size=2):
    ds = tf.data.Dataset.from_tensor_slices((inputs, targets))
    ds = ds.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

def edit_mask_images(mask_dir, func, output_dir=None):
    out_dir = output_dir or mask_dir
    create_folder(out_dir)
    exts = ('.tif')
    for fname in os.listdir(mask_dir):
        if not fname.lower().endswith(exts):
            continue
        in_path  = os.path.join(mask_dir, fname)
        mask     = cv2.imread(in_path, cv2.IMREAD_UNCHANGED)
        edited   = func(mask)
        cv2.imwrite(os.path.join(out_dir, fname), edited)
    print(f"[edit_mask_images] Applied edits to masks in {out_dir}")

if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_folder("/content/drive/MyDrive/Project_41725/files")

    batch_size = 16
    lr         = 1e-4
    epochs_num     = 100
    path_model = os.path.join("/content/drive/MyDrive/Project_41725/files", "model.h5")
    path_csv   = os.path.join("/content/drive/MyDrive/Project_41725/files", "log.csv")

    ds_path = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
    train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks = load_dataset(ds_path)

    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

    train_ds = tf_dataset(train_imgs, train_masks, batch_size=batch_size)
    val_ds   = tf_dataset(val_imgs,   val_masks,   batch_size=batch_size)
    test_ds  = tf_dataset(test_imgs,  test_masks,  batch_size=batch_size)

    # Build, compile and fit your model below:
    # model = Unet_model((Height, Width, 3))
    # model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coeff])
    # callbacks = [ModelCheckpoint(...), ...]
    # model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)


In [ ]:
import os
import cv2
import numpy as np

# TODO: update this to your actual masks directory
mask_dir = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"

# Gather all .tif/.tiff files
mask_files = [
    f for f in os.listdir(mask_dir)
    if f.lower().endswith(('.tif', '.tiff'))
]

print(f"Found {len(mask_files)} mask files in:\n  {mask_dir}\n")

suffix_count = 0

for fname in sorted(mask_files):
    path = os.path.join(mask_dir, fname)
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if mask is None:
        print(f"[ERROR] Could not read {fname}")
        continue

    stem = os.path.splitext(fname)[0]
    has_mask_suffix = stem.endswith("_mask")
    suffix_count += int(has_mask_suffix)

    unique_vals = np.unique(mask)
    print(
        f"{fname} | stem='{stem}' | '_mask' suffix? {has_mask_suffix} | "
        f"unique labels: {list(unique_vals)}"
    )

print(
    f"\nSummary:\n"
    f"  With '_mask' suffix: {suffix_count}\n"
    f"  Without suffix:      {len(mask_files) - suffix_count}"
)


Found 1738 mask files in:
  /content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks

MildDemented_00be34e4-c61c-45a8-8ee8-48e29a8adce0_jpg.rf.bc8e786abf4ec687946d6ebbbe11300e_mask.tif | stem='MildDemented_00be34e4-c61c-45a8-8ee8-48e29a8adce0_jpg.rf.bc8e786abf4ec687946d6ebbbe11300e_mask' | '_mask' suffix? True | unique labels: [np.uint8(0), np.uint8(4)]
MildDemented_01bb5650-386e-4bdd-854e-54e12030a3c0_jpg.rf.91288ed2e7955c1255997e31afe67b6f_mask.tif | stem='MildDemented_01bb5650-386e-4bdd-854e-54e12030a3c0_jpg.rf.91288ed2e7955c1255997e31afe67b6f_mask' | '_mask' suffix? True | unique labels: [np.uint8(0), np.uint8(4)]
MildDemented_01dc8d82-81d9-4344-a80e-cf981232df36_jpg.rf.f1975ff08e1b64dba4100aafd50aa3aa_mask.tif | stem='MildDemented_01dc8d82-81d9-4344-a80e-cf981232df36_jpg.rf.f1975ff08e1b64dba4100aafd50aa3aa_mask' | '_mask' suffix? True | unique labels: [np.uint8(0), np.uint8(4)]
MildDemented_02addcbc-619b-4a93-bca0-fb21e0ab7066_jpg.rf.5b10a3fa0c4f10c0b6cf689bf9766e1c_mask.t

KeyboardInterrupt: 

In [ ]:
for msk in train_masks:
    arr = cv2.imread(msk, cv2.IMREAD_UNCHANGED)
    if arr is None: continue
    if np.any(arr > 3):
        print("[INVALID]", msk, np.unique(arr))


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
)
from tensorflow.keras.models import load_model

# Hyper‑params & paths
NUM_CLASSES = 5
INPUT_SHAPE = (512, 512, 3)
EPOCHS      = 160
path_model  = "/content/drive/MyDrive/Project_41725/files/model.h5"
path_csv    = "/content/drive/MyDrive/Project_41725/files/log.csv"

# 0) kill any existing graph/state
tf.keras.backend.clear_session()

# 1) rebuild
model = build_r2attunet(INPUT_SHAPE, NUM_CLASSES, t=2)

# 2) override its internal name (strip out the '+')
model._name = "unet"

# 3) Define losses & metrics
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.squeeze(y_true, axis=-1)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=NUM_CLASSES)
    y_true_f = tf.reshape(y_true_o, [-1, NUM_CLASSES])
    y_pred_f = tf.reshape(y_pred,   [-1, NUM_CLASSES])
    inter = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    denom = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)
    dice_per_class = (2. * inter + smooth) / (denom + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    return ce + dice_loss(y_true, y_pred)

class MeanIoUArgMax(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes=NUM_CLASSES, name="mean_iou", dtype=None):
        super().__init__(num_classes=num_classes, name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)

# 4) Compile
model.compile(
    optimizer=Adam(1e-4),
    loss=combined_loss,
    metrics=[dice_coef, MeanIoUArgMax()]
)


# 5) Callbacks
callbacks = [
    ModelCheckpoint(path_model, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(path_csv , append=False),
    EarlyStopping(monitor='val_loss', patience=20),
]

# 6) (Optional) Evaluate before training
print("Before training:", dict(zip(model.metrics_names, model.evaluate(test_ds))))

# 7) Fit
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)
# 6) Inference & metric collection
EVL_SCORE_list = []
for batch_images, batch_masks in test_ds:
    preds = model.predict(batch_images)
    # for each sample in the batch:
    for i in range(len(preds)):
        y_true = batch_masks[i].numpy().squeeze(-1)
        y_pred = np.argmax(preds[i], axis=-1)
        # compute F1, Jaccard, etc. (use sklearn.metrics)
        f1 = f1_score(y_true.flatten(), y_pred.flatten(), average='macro', zero_division=0)
        j  = jaccard_score(y_true.flatten(), y_pred.flatten(), average='macro', zero_division=0)
        r  = recall_score(y_true.flatten(), y_pred.flatten(), average='macro', zero_division=0)
        p  = precision_score(y_true.flatten(), y_pred.flatten(), average='macro', zero_division=0)
        img_name = "batch{}_idx{}".format(batch_index, i)  # or however you track filenames
        EVL_SCORE_list.append((img_name, f1, j, r, p))

# 7) Save per-image metrics
df_scores = pd.DataFrame(EVL_SCORE_list,
                         columns=["Image","F1","Jaccard","Recall","Precision"])
os.makedirs(os.path.dirname(score_csv), exist_ok=True)
df_scores.to_csv(score_csv, index=False)
print("Wrote", score_csv)


11/11 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - dice_coef: 0.1104 - loss: 2.4861 - mean_iou: 0.1263
Before training: {'loss': 2.4819869995117188, 'compile_metrics': 0.11027178913354874}
Epoch 1/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 1000ms/step - dice_coef: 0.3153 - loss: 1.2132 - mean_iou: 0.2984
Epoch 1: val_loss improved from inf to 2.02465, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step - dice_coef: 0.3158 - loss: 1.2098 - mean_iou: 0.2989 - val_dice_coef: 0.1566 - val_loss: 2.0247 - val_mean_iou: 0.1455 - learning_rate: 1.0000e-04
Epoch 2/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 895ms/step - dice_coef: 0.3982 - loss: 0.7621 - mean_iou: 0.3790
Epoch 2: val_loss did not improve from 2.02465
76/76 ━━━━━━━━━━━━━━━━━━━━ 75s 959ms/step - dice_coef: 0.3984 - loss: 0.7617 - mean_iou: 0.3793 - val_dice_coef: 0.1011 - val_loss: 3.0492 - val_mean_iou: 0.0368 - learning_rate: 1.0000e-04
Epoch 3/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 774ms/step - dice_coef: 0.4492 - loss: 0.6705 - mean_iou: 0.4211
Epoch 3: val_loss did not improve from 2.02465
76/76 ━━━━━━━━━━━━━━━━━━━━ 64s 838ms/step - dice_coef: 0.4494 - loss: 0.6702 - mean_iou: 0.4213 - val_dice_coef: 0.0312 - val_loss: 13.6767 - val_mean_iou: 0.0158 - learning_rate: 1.0000e-04
Epoch 4/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 775ms/step - dice_coef: 0.5154 - loss: 0.5811 - mean_iou: 0.4756
Epoch 4: val_los

76/76 ━━━━━━━━━━━━━━━━━━━━ 69s 905ms/step - dice_coef: 0.6301 - loss: 0.4431 - mean_iou: 0.5707 - val_dice_coef: 0.3737 - val_loss: 1.3763 - val_mean_iou: 0.3219 - learning_rate: 1.0000e-04
Epoch 7/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 897ms/step - dice_coef: 0.6190 - loss: 0.4603 - mean_iou: 0.5640
Epoch 7: val_loss improved from 1.37631 to 0.77462, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - dice_coef: 0.6195 - loss: 0.4598 - mean_iou: 0.5643 - val_dice_coef: 0.5137 - val_loss: 0.7746 - val_mean_iou: 0.4484 - learning_rate: 1.0000e-04
Epoch 8/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 887ms/step - dice_coef: 0.6824 - loss: 0.3802 - mean_iou: 0.6369
Epoch 8: val_loss improved from 0.77462 to 0.48651, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - dice_coef: 0.6827 - loss: 0.3799 - mean_iou: 0.6370 - val_dice_coef: 0.6537 - val_loss: 0.4865 - val_mean_iou: 0.5907 - learning_rate: 1.0000e-04
Epoch 9/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 889ms/step - dice_coef: 0.7049 - loss: 0.3512 - mean_iou: 0.6468
Epoch 9: val_loss improved from 0.48651 to 0.46050, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


...

76/76 ━━━━━━━━━━━━━━━━━━━━ 70s 921ms/step - dice_coef: 0.9279 - loss: 0.0867 - mean_iou: 0.9318 - val_dice_coef: 0.8309 - val_loss: 0.2259 - val_mean_iou: 0.7854 - learning_rate: 1.0000e-06
Epoch 78/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 891ms/step - dice_coef: 0.9280 - loss: 0.0865 - mean_iou: 0.9324
Epoch 78: val_loss did not improve from 0.22589
76/76 ━━━━━━━━━━━━━━━━━━━━ 74s 954ms/step - dice_coef: 0.9280 - loss: 0.0865 - mean_iou: 0.9321 - val_dice_coef: 0.8309 - val_loss: 0.2261 - val_mean_iou: 0.7853 - learning_rate: 1.0000e-06
Epoch 79/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 774ms/step - dice_coef: 0.9281 - loss: 0.0863 - mean_iou: 0.9326
Epoch 79: val_loss did not improve from 0.22589
76/76 ━━━━━━━━━━━━━━━━━━━━ 64s 838ms/step - dice_coef: 0.9282 - loss: 0.0863 - mean_iou: 0.9324 - val_dice_coef: 0.8310 - val_loss: 0.2259 - val_mean_iou: 0.7854 - learning_rate: 1.0000e-06
Epoch 80/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 774ms/step - dice_coef: 0.9283 - loss: 0.0861 - mean_iou: 0.9329
Epoch 80: 

76/76 ━━━━━━━━━━━━━━━━━━━━ 69s 914ms/step - dice_coef: 0.9285 - loss: 0.0859 - mean_iou: 0.9329 - val_dice_coef: 0.8310 - val_loss: 0.2259 - val_mean_iou: 0.7854 - learning_rate: 1.0000e-06
Epoch 82/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 895ms/step - dice_coef: 0.9286 - loss: 0.0857 - mean_iou: 0.9335
Epoch 82: val_loss did not improve from 0.22587

Epoch 82: ReduceLROnPlateau reducing learning rate to 1e-07.
76/76 ━━━━━━━━━━━━━━━━━━━━ 73s 958ms/step - dice_coef: 0.9287 - loss: 0.0857 - mean_iou: 0.9332 - val_dice_coef: 0.8309 - val_loss: 0.2262 - val_mean_iou: 0.7852 - learning_rate: 1.0000e-06
Epoch 83/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 774ms/step - dice_coef: 0.9288 - loss: 0.0855 - mean_iou: 0.9336
Epoch 83: val_loss improved from 0.22587 to 0.22553, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 70s 927ms/step - dice_coef: 0.9288 - loss: 0.0855 - mean_iou: 0.9333 - val_dice_coef: 0.8315 - val_loss: 0.2255 - val_mean_iou: 0.7860 - learning_rate: 1.0000e-07
Epoch 84/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 892ms/step - dice_coef: 0.9289 - loss: 0.0853 - mean_iou: 0.9339
Epoch 84: val_loss improved from 0.22553 to 0.22543, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - dice_coef: 0.9290 - loss: 0.0853 - mean_iou: 0.9336 - val_dice_coef: 0.8315 - val_loss: 0.2254 - val_mean_iou: 0.7861 - learning_rate: 1.0000e-07
Epoch 85/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 896ms/step - dice_coef: 0.9290 - loss: 0.0852 - mean_iou: 0.9340
Epoch 85: val_loss did not improve from 0.22543
76/76 ━━━━━━━━━━━━━━━━━━━━ 74s 959ms/step - dice_coef: 0.9290 - loss: 0.0852 - mean_iou: 0.9337 - val_dice_coef: 0.8315 - val_loss: 0.2255 - val_mean_iou: 0.7861 - learning_rate: 1.0000e-07
Epoch 86/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 774ms/step - dice_coef: 0.9290 - loss: 0.0852 - mean_iou: 0.9341
Epoch 86: val_loss did not improve from 0.22543
76/76 ━━━━━━━━━━━━━━━━━━━━ 64s 838ms/step - dice_coef: 0.9291 - loss: 0.0852 - mean_iou: 0.9338 - val_dice_coef: 0.8315 - val_loss: 0.2256 - val_mean_iou: 0.7860 - learning_rate: 1.0000e-07
Epoch 87/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 775ms/step - dice_coef: 0.9290 - loss: 0.0852 - mean_iou: 0.9341
Epoch 87: val

NameError: name 'f1_score' is not defined

#Test


In [ ]:
from google.colab import drive
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files

# 1) Csv
import pandas as pd

# suppose you build EVL_SCORE_list as [(img_name, f1, jaccard, recall, precision), …]
df = pd.DataFrame(
    EVL_SCORE_list,
    columns=["Image", "F1", "Jaccard", "Recall", "Precision"]
)
df.to_csv(
    '/content/drive/MyDrive/DSP Project/files/score.csv',
    index=False
)
model_path = "/content/drive/MyDrive/Project_41725/files/LinkNet/model.h5"

# 3) Load model for inference only—skip compile to avoid the TypeError
model = load_model(
    model_path,
    custom_objects={
        'dice_coefficient': dice_coefficient,
        'dice_coefficient_loss': dice_coefficient_loss
    },
    compile=False
)

def predict_and_show(img_path, model, input_size=(512,512)):
    gray    = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    resized = cv2.resize(gray, input_size)
    x       = np.stack([resized]*3, -1).astype(np.float32) / 255.0

    # Run model
    pred = model.predict(x[None])[0]         # ➔ (H, W, num_classes)
    seg  = pred.argmax(-1).astype(np.uint8)  # ➔ (H, W)

    # Compute and print class percentages
    unique, counts = np.unique(seg, return_counts=True)
    total = seg.size
    percentages = {int(cls): float(cnt/total*100) for cls, cnt in zip(unique, counts)}
    print(f"\nClass percentages for {img_path}:")
    for cls, pct in percentages.items():
        print(f"  Class {cls}: {pct:.2f}%")

    # Display images
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.title("Input")
    plt.imshow(resized, cmap='gray')
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.title("Segmentation Overlay")
    plt.imshow(resized, cmap='gray', alpha=0.5)
    plt.imshow(seg, cmap='jet',   alpha=0.5)
    plt.axis('off')
    plt.show()

# 4) Upload & run on uploaded files
uploaded = files.upload()
for fn in uploaded:
    predict_and_show(fn, model)
    print("Num output channels:", model.output_shape[-1])


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# -- ensure output folder exists --
out_dir = '/content/drive/MyDrive/DSP Project/files'
os.makedirs(out_dir, exist_ok=True)

# 1) Load evaluation results
df = pd.read_csv(os.path.join(out_dir, 'score.csv'))

# 2) (Optional) Inspect to confirm non-zero values
print(df.head(), '\n', df.dtypes)

# 3) Your plots…
plt.figure(figsize=(10,6))
for m in ['F1','Jaccard','Recall','Precision']:
    plt.hist(df[m], bins=20, alpha=0.5, label=m)
plt.legend(); plt.show()

# 4) Compute & save summary stats
summary = pd.DataFrame({
    'Metric':  ['F1','Jaccard','Recall','Precision'],
    'Mean':    df[['F1','Jaccard','Recall','Precision']].mean().values,
    'Std Dev': df[['F1','Jaccard','Recall','Precision']].std().values
})
summary_path = os.path.join(out_dir, 'summary.csv')
summary.to_csv(summary_path, index=False)
print(f"Saved summary to {summary_path}:\n", summary)


#Plot and Checks

In [ ]:
import os
import glob
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ace_tools import display_dataframe_to_user

# 1) Point to your mask directory
mask_dir = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"  # <— update this path

# 2) Collect all mask file paths
mask_paths = glob.glob(os.path.join(mask_dir, "*"))

# 3) Initialize pixel count dictionary
num_classes = 5  # update if different
pixel_counts = {cls: 0 for cls in range(num_classes)}

# 4) Count pixels per class across all masks
for path in mask_paths:
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    # If mask has channel dimension, reduce to single channel
    if mask.ndim == 3:
        mask = mask[..., 0]
    flat = mask.flatten()
    for cls in range(num_classes):
        pixel_counts[cls] += np.sum(flat == cls)

# 5) Compute total and percentages
total_pixels = sum(pixel_counts.values())
percentages = {cls: (count / total_pixels * 100) for cls, count in pixel_counts.items()}

# 6) Prepare DataFrame
df = pd.DataFrame({
    "Class": list(pixel_counts.keys()),
    "Pixel Count": list(pixel_counts.values()),
    "Percentage (%)": [percentages[cls] for cls in pixel_counts]
})

# 7) Display table to user
display_dataframe_to_user("Class Distribution", df)

# 8) Plot percentages
plt.figure(figsize=(8, 5))
plt.bar(df["Class"].astype(str), df["Percentage (%)"])
plt.title("Mask Pixel Percentage by Class")
plt.xlabel("Class")
plt.ylabel("Percentage of Total Mask Pixels")
plt.show()
